# Getting started with the RVAI SDK

This tutorial will get you started with the RVAI SDK.

A dummy pipeline will be created that takes two float inputs and calculates both the sum and multiplication.

Documentation on the Cell API and Pipeline API can be found here: https://base.rvai.dev.

## Installation

First install the required the sdk python packages.
Simplest way is to install the 'rvai' meta package which will install all SDK related packages. Access to our devpi server is required.


In [ ]:
!pip install rvai==1.1.0rc51 pygraphviz

## Creating the cells

First we will create a SummatorCell and a MultiplatorCell that both accept 2 floating point inputs and generate a floating point output.

In [ ]:
# import base classes
from dataclasses import dataclass, field
from rvai.base.cell import Cell, TrainableCell, cell
from rvai.base.data import (
    Annotations,
    Inputs,
    Outputs,
    Parameters,
    ProcessedParameters,
)
from rvai.base.context import InferenceContext, ParameterContext
from rvai.types import Float

### SummatorCell

In [ ]:
# Inputs
@dataclass
class SummatorInputs(Inputs):
    op_1: Float = Inputs.field(
        name="Operand 1", description="A floating point."
    )
    op_2: Float = Inputs.field(
        name="Operand 2", description="A floating point."
    )

# Outputs
@dataclass
class SummatorOutputs(Outputs):
    sum: Float = Outputs.field(
        name="Sum", description="Sum of operand 1 and 2."
    )

# Parameters
@dataclass
class SummatorParameters(Parameters):
    threshold: Float = Parameters.field(
        default=Float(100.0),
        name="Threshold", description="Threshold of the sum."
    )

# Cell
@cell
class SummatorCell(Cell):

    @classmethod
    def process_parameters(
        cls, context: ParameterContext, parameters: SummatorParameters
    ) -> SummatorParameters:
        return parameters

    @classmethod
    def call(
        cls, context: InferenceContext, parameters: SummatorParameters, inputs: SummatorInputs,
    ) -> SummatorOutputs:
        return SummatorOutputs(sum=Float(min(inputs.op_1 + inputs.op_2, parameters.threshold)))


### MultiplierCell

In [ ]:
# Inputs
@dataclass
class MultiplierInputs(Inputs):
    op_1: Float = Inputs.field(
        name="Operand 1", description="A floating point."
    )
    op_2: Float = Inputs.field(
        name="Operand 2", description="A floating point."
    )

# Outputs
@dataclass
class MultiplierOutputs(Outputs):
    product: Float = Outputs.field(
        name="Product", description="Product of operand 1 and 2."
    )

# Parameters
@dataclass
class MultiplierParameters(Parameters):
    pass

# Cell
@cell
class MultiplierCell(Cell):

    @classmethod
    def process_parameters(
        cls, context: ParameterContext, parameters: MultiplierParameters
    ) -> MultiplierParameters:
        return parameters

    @classmethod
    def call(
        cls, context: InferenceContext, parameters: MultiplierParameters, inputs: MultiplierInputs,
    ) -> MultiplierOutputs:
        return MultiplierOutputs(product=Float(inputs.op_1 * inputs.op_2))

## Creating a pipeline

In [ ]:
from rvai.base.pipeline import DeclarativePipeline, PipelineCells, PipelineFactory, pipeline

In [ ]:
# imperative
def generate_pipeline():
    # create pipeline
    p = PipelineFactory(name="MyPipeline")
    
    # create cells and add them to pipeline
    sum_cell = SummatorCell()    
    mul_cell = MultiplierCell()    
    p.add_cell(ref="sum", cell=sum_cell)
    p.add_cell(ref="mul", cell=mul_cell)
    
    # setup connections
    p.declare_input(ref="a", input=sum_cell.inputs.op_1)
    p.declare_input(ref="b", input=sum_cell.inputs.op_2)
    p.declare_input(ref="a", input=mul_cell.inputs.op_1)
    p.declare_input(ref="b", input=mul_cell.inputs.op_2)
    
    p.declare_output(ref="sum", output=sum_cell.outputs.sum)
    p.declare_output(ref="product", output=mul_cell.outputs.product)
    
    return p.build()

# declarative
class MyPipelineCells(PipelineCells):
    sum: SummatorCell
    mult: MultiplierCell

@pipeline
class MyPipeline(DeclarativePipeline):

    cells = MyPipelineCells
    inputs = {
        "a": [cells.sum.inputs.op_1, cells.mult.inputs.op_1],
        "b": [cells.sum.inputs.op_2, cells.mult.inputs.op_2],
    }
    outputs = {
        "sum": cells.sum.outputs.sum,
        "product": cells.mult.outputs.product,
    }

In [ ]:
pipeline = MyPipeline.build()
# %matplotlib inline
pipeline.show()

## Initializing a runtime

A runtime is required to run the pipeline. In this case we use the DebugRuntime which is a simple runtime that runs the cells sequentially.

We also start an inference service for the pipeline.

In [ ]:
from rvai.base.runtime import Inference, init

# init debug runtime
rt = init("ray")

# create inference task
inference = Inference(pipeline=pipeline)
proc = rt.start_inference(inference)

Requesting a prediction:

In [ ]:
# get prediction
pred = proc.predict({"a": Float(2.0), "b": Float(3.0)})
print(pred.result())
pred = proc.predict({"a": Float(4.0), "b": Float(5.0)})
print(pred.result())

Update parameters:

In [ ]:
# update parameters for a certain state id
proc.set_parameters(parameters={"sum": SummatorParameters(threshold=Float(1.0))})
proc.set_parameters(parameters={"sum": SummatorParameters(threshold=Float(10.0))}, sid="stream-1")

# get prediction on default state
pred = proc.predict({"a": Float(10.0), "b": Float(30.0)})
print("default parameters:", pred.result())

# get prediction on 'stream-1' state
pred = proc.predict({"a": Float(10.0), "b": Float(30.0)}, sid="stream-1")
print("stream-1 parameters:", pred.result())

Stop the process and runtime.
In fact, stopping the runtime would be sufficient as that stops all processes as well.

In [ ]:
proc.stop()
rt.stop()